In [1]:
import shutil
import os

In [2]:
pwd = os.getcwd()
#pwd += "/rheo_bench/channel/"
pwd

'/home/epalermo/test/rheo_bench/grad_v/2D'

In [3]:
#parameter grids
var_sf_grid = {"0.05":"20","0.1":"10","0.2":"5","0.25":"4","0.5":"2"}
var_kernel_grid = {"CRK0":"0","CRK1":"1","CRK2":"2","Quintic":"5"}
#var_hd_grid = {"1.5":"1p5","2.0":"2p0","2.5":"2p5","3.5":"3p5","4.5":"4p5"}
var_hd_grid = {"3.5":"3p5"}



# Generate LAMMPS Input Scripts

In [4]:
#Generate lammps files
filenames = []
for sf in var_sf_grid:
    for kernel in var_kernel_grid:
        for hd in var_hd_grid:
            run_code = "1_"+var_kernel_grid[kernel]+"_"+var_sf_grid[sf]+"_"+var_hd_grid[hd] #shifting on (1) is constant
            filename = "lammps_in__"+run_code
            file = shutil.copyfile("lammps_in.template",pwd+"/"+filename+".lmp")
            file_in = open(file,"rt")
            file_text = file_in.read()

            #Replace variables
            file_text = file_text.replace("var_sf",sf)
            file_text = file_text.replace("var_hd",hd)
            file_text = file_text.replace("var_kernel",kernel)
            file_text = file_text.replace("var_filename",run_code)

            file_out = open(file,"wt")
            file_out.write(file_text)
            file_out.close()
            
            filenames.append(filename+".lmp")

In [5]:
filenames

['lammps_in__1_0_20_3p5.lmp',
 'lammps_in__1_1_20_3p5.lmp',
 'lammps_in__1_2_20_3p5.lmp',
 'lammps_in__1_5_20_3p5.lmp',
 'lammps_in__1_0_10_3p5.lmp',
 'lammps_in__1_1_10_3p5.lmp',
 'lammps_in__1_2_10_3p5.lmp',
 'lammps_in__1_5_10_3p5.lmp',
 'lammps_in__1_0_5_3p5.lmp',
 'lammps_in__1_1_5_3p5.lmp',
 'lammps_in__1_2_5_3p5.lmp',
 'lammps_in__1_5_5_3p5.lmp',
 'lammps_in__1_0_4_3p5.lmp',
 'lammps_in__1_1_4_3p5.lmp',
 'lammps_in__1_2_4_3p5.lmp',
 'lammps_in__1_5_4_3p5.lmp',
 'lammps_in__1_0_2_3p5.lmp',
 'lammps_in__1_1_2_3p5.lmp',
 'lammps_in__1_2_2_3p5.lmp',
 'lammps_in__1_5_2_3p5.lmp']

# Write the Jobscript

### Solo

In [10]:
account="FY150028"
job_name="TestChannel"
time="24:0:0"
partition="batch"
nodes="1"
ntasks_per_node="16"
run_command = "mpirun -np 16 /ascldap/users/etpaler/dev/lammps_rheo/lammps/build/lmp -in "

In [11]:
with open("jobscript","w") as jobscript:
        #---shebang---#
        jobscript.write("#!/bin/bash\n\n")

        #---slurm commands---#
        jobscript.write("#SBATCH --account={}\n".format(account))
        jobscript.write("#SBATCH --job-name={}\n".format(job_name))
        jobscript.write("#SBATCH --time={}\n".format(time))
        jobscript.write("#SBATCH --partition={}\n".format(partition))
        jobscript.write("#SBATCH --nodes={}\n".format(nodes))
        jobscript.write("#SBATCH --ntasks-per-node={}\n\n".format(ntasks_per_node))
        
        jobscript.write("module load cmake/3.20.3 intel/18.0.0.128 openmpi-intel/4.1 mkl/18.0.0.128\n\n")
        
        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")

### WSL

In [6]:
run_command = "mpirun -np 4 ~/dev/rlammps_inlet/build/lmp -in "

with open("jobscript","w") as jobscript:
        jobscript.write("#!/bin/bash\n\n")

        for filename in filenames:
            jobscript.write(run_command+filename+"\n")
            
        jobscript.write("\n")